In [0]:
test_case_1 = sc.parallelize([("A","B"), ("B","C"), ("B","D"), ("D","E"), ("F","G"), ("G", "H")])
test_case_1.collect()

Out[1]: [('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

In [0]:
#Algo 1
def ccf_iterate(rdd, counter_pair):
    # Map
    rdd = rdd.map(lambda x: [(x[0], x[1]),(x[1], x[0])]).flatMap(lambda x: x)
    rdd = rdd.reduceByKey(lambda x, y: x  + y).map(lambda x: (x[0], list(x[1])))

    minEachKey = rdd.map(lambda x: (x[0], x[1], min(x[1]))).map(lambda x: (x[0], x[1], min(x[2], x[0])))
    emit_1 = minEachKey.filter(lambda x: x[2] < x[0]).map(lambda x: (x[0], x[2]))
    emit_2 = minEachKey.filter(lambda x: x[2] < x[0]).map(lambda x: (x[0], [y for y in x[1] if y != x[2]], x[2])) # remove min in valueList

    counter_pair += emit_2.map(lambda x: len(x[1])).reduce(lambda x, y: x+ y) # Increase counter
    emit_2 = emit_2.filter(lambda x: len(x[1]) != 0).map(lambda x: [(y, x[2]) for y in x[1]]).flatMap(lambda x: x)

    emit = emit_1.union(emit_2)
    # CCF-dedub
    emit = emit.map(lambda x: (x, [])).reduceByKey(lambda x, y: x).map(lambda x: x[0])
    return emit, counter_pair

In [0]:
a = test_case_1.map(lambda x: [(x[0], x[1]),(x[1], x[0])]).flatMap(lambda x: x)
a.collect()

Out[3]: [('A', 'B'),
 ('B', 'A'),
 ('B', 'C'),
 ('C', 'B'),
 ('B', 'D'),
 ('D', 'B'),
 ('D', 'E'),
 ('E', 'D'),
 ('F', 'G'),
 ('G', 'F'),
 ('G', 'H'),
 ('H', 'G')]

In [0]:
graph = a.reduceByKey(lambda x, y: x  + y).map(lambda x: (x[0], list(x[1])))
graph.collect()

Out[4]: [('B', ['A', 'C', 'D']),
 ('H', ['G']),
 ('C', ['B']),
 ('A', ['B']),
 ('F', ['G']),
 ('D', ['B', 'E']),
 ('E', ['D']),
 ('G', ['F', 'H'])]

In [0]:
minEachKey = graph.map(lambda x: (x[0], x[1], min(x[1]))).map(lambda x: (x[0], x[1], min(x[2], x[0])))
minEachKey.collect()

Out[5]: [('B', ['A', 'C', 'D'], 'A'),
 ('H', ['G'], 'G'),
 ('C', ['B'], 'B'),
 ('A', ['B'], 'A'),
 ('F', ['G'], 'F'),
 ('D', ['B', 'E'], 'B'),
 ('E', ['D'], 'D'),
 ('G', ['F', 'H'], 'F')]

In [0]:
emit_1 = minEachKey.filter(lambda x: x[2] < x[0]).map(lambda x: (x[0], x[2]))
emit_1.collect()

Out[6]: [('B', 'A'), ('H', 'G'), ('C', 'B'), ('D', 'B'), ('E', 'D'), ('G', 'F')]

In [0]:
tmp = minEachKey.filter(lambda x: x[2] < x[0]).map(lambda x: (x[0], [y for y in x[1] if y != x[2]], x[2]))
emit_2 = minEachKey.filter(lambda x: x[2] < x[0]).map(lambda x: (x[0], [y for y in x[1] if y != x[2]], x[2])).filter(lambda x: len(x[1]) != 0).map(lambda x: [(y, x[2]) for y in x[1]]).flatMap(lambda x: x)
tmp.map(lambda x: len(x[1])).reduce(lambda x, y: x+ y)

Out[37]: 4

In [0]:
emit = emit_1.union(emit_2)
emit.collect()

Out[10]: [('B', 'A'),
 ('H', 'G'),
 ('C', 'B'),
 ('D', 'B'),
 ('E', 'D'),
 ('G', 'F'),
 ('C', 'A'),
 ('D', 'A'),
 ('E', 'B'),
 ('H', 'F')]

In [0]:
emit.map(lambda x: (x, [])).reduceByKey(lambda x, y: x).map(lambda x: x[0]).collect()


Out[28]: [('B', 'A'),
 ('H', 'F'),
 ('D', 'A'),
 ('C', 'A'),
 ('D', 'B'),
 ('E', 'B'),
 ('C', 'B'),
 ('G', 'F'),
 ('E', 'D'),
 ('H', 'G')]

[(8, 6), (8, 7), (2, 1), (3, 1), (3, 2), (5, 4), (4, 1), (5, 2), (4, 2), (7, 6)]


In [0]:
test_case_11 = sc.parallelize([(1,2), (2,3), (2,4), (4,5), (6,7), (7, 8)])

In [0]:
new_pairs = 0
def CFF_iterate(rdd):
    rdd_inv = rdd.map(lambda x: (x[1], x[0])) 
    graph = rdd.union(rdd_inv) # emit pairs
    graph = graph.groupByKey().mapValues(list).partitionBy(100).cache() # group by key and get values as a list
    graph = graph.map(lambda x: (int(x[0]), list(map(int, x[1])))) # Convert string to integer
    graph = graph.map(lambda x: (x[0], x[1], min(x[1]))) # extract min
    graph = graph.filter(lambda x: x[0] > x[2]) # filter out vid < amin
    graph = graph.map(lambda x: (x[0], [val for val in x[1] if val != x[2]], x[2])) # remove min from list
    
    new_pairs = graph.map(lambda x: len(x[1])).sum()  # get number of new pairs created

    emit_1 = graph.map(lambda x: (x[0], x[2]))
    emit_2 = graph.flatMap(lambda x: [(val, x[2]) for val in x[1]])
    emit_tot = emit_1.union(emit_2) # return all pairs (CFF-Dedup)
    
    new = emit_tot.map(lambda x: ((x[0], x[1]), [])) 
    new = new.groupByKey().mapValues(list)
    new = new.map(lambda x: (x[0][0], x[0][1]))
    return new, new_pairs

In [0]:
CFF_iterate(test_case_11)[0].collect()

Out[15]: [(8, 6),
 (8, 7),
 (2, 1),
 (3, 1),
 (3, 2),
 (5, 4),
 (4, 1),
 (5, 2),
 (4, 2),
 (7, 6)]

In [0]:
test_case_2 = sc.parallelize([(1,2), (2,3), (2,4), (4,5), (6,7), (7, 8), (1, 2)])

In [0]:
test_case_2.map(lambda x: (x, [])).reduceByKey(lambda x, y: x).collect()

Out[26]: [((1, 2), []),
 ((4, 5), []),
 ((2, 4), []),
 ((2, 3), []),
 ((6, 7), []),
 ((7, 8), [])]

In [0]:
res = ccf_iterate(test_case_1, 0)
res[0].collect()

Out[40]: [('B', 'A'),
 ('H', 'F'),
 ('D', 'A'),
 ('C', 'A'),
 ('D', 'B'),
 ('E', 'B'),
 ('C', 'B'),
 ('G', 'F'),
 ('E', 'D'),
 ('H', 'G')]

In [0]:
res[1]

Out[41]: 4

In [0]:
#main
new_pair = 1
rdd = test_case_1
while (new_pair > 0):
    rdd, new_pair = ccf_iterate(rdd, 0)
    print("new_pair = ", new_pair)


new_pair =  4
new_pair =  9
new_pair =  4
new_pair =  0


In [0]:
rdd.collect()

Out[46]: [('D', 'A'), ('E', 'A'), ('H', 'F'), ('G', 'F'), ('B', 'A'), ('C', 'A')]